In [1]:
!pip install transformers torch nltk rouge_score

In [1]:
import math
import random
import sys
import os
# from framework.utilities.models import inference    
from framework.utilities.metrics import calculate_bleu
import fastText
from contextlib import contextmanager
import time
import json

In [2]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [3]:
import pickle
import pandas as pd

with open('data/combined_outputs.pkl', 'rb') as file:
    combined_outputs = pickle.load(file)

df_combined_outputs = pd.DataFrame(combined_outputs)
df_combined_outputs.head()

,wmt14,cnn_dailymail,gsm8k
0,"{'input': 'Eine republikanische Strategie, um ...","{'input': '(CNN)Share, and your gift will be m...",{'input': 'Janet’s ducks lay 16 eggs per day. ...
1,{'input': 'Die Führungskräfte der Republikaner...,"{'input': '(CNN)On the 6th of April 1996, San ...",{'input': 'A robe takes 2 bolts of blue fiber ...
2,{'input': 'Allerdings hält das Brennan Center ...,{'input': '(CNN)French striker Bafetimbi Gomis...,{'input': 'Josh decides to try flipping a hous...
3,{'input': 'Die Rechtsanwälte der Republikaner ...,{'input': '(CNN)It was an act of frustration p...,{'input': 'James decides to run 3 sprints 3 ti...
4,{'input': 'Eins ist sicher: diese neuen Bestim...,{'input': '(CNN)A Pennsylvania community is pu...,"{'input': 'Every day, Wendi feeds each of her ..."


In [4]:
df_combined_outputs.size

3000

In [5]:
df_combined_outputs["wmt14"][0]

{'input': 'Eine republikanische Strategie, um der Wiederwahl von Obama entgegenzutreten',
 'label': 'A Republican strategy to counter the re-election of Obama',
 '7b': ["A Republican Strategy to Avoid Obama's Reelection"],
 'tiny': ["A Republican Strategy to Avoid Obama's Reelection"],
 '13b': ["A republican strategy to counter Obama's re-election"]}

In [6]:
def algorithm(T, c):
    l_predictor, s_predictor = None, None
    results = []
    
    for t in range(0, T):
        print(f"Index: {t}")
        t_i = df_combined_outputs["wmt14"][t]["input"]
        t_e = df_combined_outputs["wmt14"][t]["label"]

        p_t = min(1, c/math.sqrt(t))
        X_t = Bernoulli(p_t)
        
        if X_t == 1:
            t_c = getResults(t, t_i, t_e)
            l_predictor, s_predictor = sgdStep(t_c)
            l_predictor, s_predictor = checkpoint(l_predictor, s_predictor, t, p_t)

        output, t_acc = queryBest(t, t_i, t_e, l_predictor, s_predictor)
        print(t_acc)
        results.append({
            'iteration': t,
            'input': t_i,
            'best_output': output,
            'chosen_model_accuracy': t_acc
        })

    return results

In [7]:
def Bernoulli(p_t):
    return random.random() < p_t 

In [8]:
def queryBest(t, t_i, t_e, l_predictor, s_predictor):
    l_acc, s_acc = predict(t_i, l_predictor, s_predictor)
    if l_acc > s_acc: # use larger model, e.g. 7b llama
        output = df_combined_outputs["wmt14"][t]["13b"]
    else: # use smaller model for all other cases, e.g. 3b llama
        output =  df_combined_outputs["wmt14"][t]["tiny"]

    return output, calculate_bleu(output[0], t_e)

In [9]:
def sgdStep(t_c):
    with open("fasttext_large.txt", "w") as f:
        f.write(f"__label__{t_c['large_model_accuracy']} {t_c['input_text']}\n")
    with open("fasttext_small.txt", "w") as f:
        f.write(f"__label__{t_c['small_model_accuracy']} {t_c['input_text']}\n")

    if os.path.exists("large_predictor.bin"):
        large_model_predictor = fastText.train_supervised(input="fasttext_large.txt", epoch=1, lr=1.0, wordNgrams=2, inputModel="large_predictor.bin")
    else:
        large_model_predictor = fastText.train_supervised(input="fasttext_large.txt", epoch=1, lr=1.0, wordNgrams=2)

    if os.path.exists("small_predictor.bin"):
        small_model_predictor = fastText.train_supervised(input="fasttext_small.txt", epoch=1, lr=1.0, wordNgrams=2, inputModel="small_predictor.bin")
    else:
        small_model_predictor = fastText.train_supervised(input="fasttext_small.txt", epoch=1, lr=1.0, wordNgrams=2)

    return large_model_predictor, small_model_predictor

In [10]:
def predict(text, l_predictor, s_predictor):
    l_predicted_label = l_predictor.predict(text)[0][0]
    l_predicted_accuracy = int(round(float(l_predicted_label.replace('__label__', ''))))

    s_predicted_label = s_predictor.predict(text)[0][0]
    s_predicted_accuracy = int(round(float(s_predicted_label.replace('__label__', ''))))

    return l_predicted_accuracy, s_predicted_accuracy

In [11]:
def getResults(t, t_i, t_e):
    s_output = df_combined_outputs["wmt14"][t]["tiny"]
    l_output = df_combined_outputs["wmt14"][t]["13b"]
    
    # WMT14
    s_acc = calculate_bleu(s_output[0], t_e)
    l_acc = calculate_bleu(l_output[0], t_e)

    return {
        'input_text': t_i,
        'large_model_accuracy': l_acc,
        'small_model_accuracy': s_acc
    }

In [12]:
def checkpoint(l_predictor, s_predictor, t, p_t):
    if p_t > 0.5:
        if t % int(math.sqrt(t)) == 0:
            print("Checkpoint")
            l_predictor, s_predictor = save_models(l_predictor, s_predictor)
    else:
        if t % max(1, int(1 / math.sqrt(t))) == 0:
            print("Checkpoint")
            l_predictor, s_predictor = save_models(l_predictor, s_predictor)

    return l_predictor, s_predictor

In [13]:
def save_models(l_predictor, s_predictor):
    if l_predictor:
        l_predictor.save_model("large_predictor.bin")
    if s_predictor:
        s_predictor.save_model("small_predictor.bin")

    return l_predictor, s_predictor

In [1]:
results = algorithm(999, 5)
print(results)

NameError: name 'algorithm' is not defined

In [8]:
with open('data/wmt14_results.json', 'w') as f:
    json.dump(results, f, indent=4)

In [2]:
import json

with open('data/wmt14_results.json', 'r') as f:
    data = json.load(f)

for result in data[:10]:
    print(result['chosen_model_accuracy'])

0.05312123894060139
0.2982895720543635
0.2830956597218095
0.708587641194393
0.06775875292600428
0.20377924119043478
0.15209404090446677
0.3033668865762665
0.5131275135405763
0.5028248236576276


In [3]:
total = 0

for result in data:
    total += result['chosen_model_accuracy']

total / 999

0.18766023212954513